## REFPROP 10.0 conversion

As of ``teqp`` version 0.19.0, it is possible to read in the .FLD and HMX.BNC of [NIST REFPROP 10.0](https://www.nist.gov/srd/refprop) and load them into ``teqp`` multifluid models. There are two approaches; either you can pass paths to the files of interest, or you can load them into JSON once, and pass the converted JSON back into teqp's ``make_model`` function.

The conversion code is uses that of [REFPROP-interop](https://github.com/ianhbell/REFPROP-interop) and the fluid file format of [CoolProp](https://github.com/coolprop/coolprop) is used.

The example is based on the interaction parameters provided in the supporting information of the paper [Mixture Model for Refrigerant Pairs R-32/1234yf, R-32/1234ze(E), R-1234ze(E)/227ea, R-1234yf/152a, and R-125/1234yf](https://doi.org/10.1063/5.0135368) by Ian Bell

In [ ]:
import json
import teqp
teqp.__version__

In [ ]:
# The first approach, we just pass paths to the files, they live in the folder 
# containing this notebook, and teqp does the conversion on the fly
jsimple = {
    'kind': 'multifluid',
    'model': {
        'HMX.BNC': 'HMX.BNC',
        'components': ['R152A.FLD', 'NEWR1234YF.FLD'],
    }
}
model = teqp.make_model(jsimple)

In [ ]:
%timeit teqp.make_model(jsimple)

In [ ]:
# Convert each of the FLD files to JSON
FLD0 = teqp.convert_FLD('R152A.FLD', name='R152A')
FLD1 = teqp.convert_FLD('NEWR1234YF.FLD', name='R1234YF')
BIP, DEP = teqp.convert_HMXBNC('HMX.BNC')

In [ ]:
jconverted = {
    "kind": "multifluid",
    "model": {
        "components": [FLD0, FLD1],
        "BIP": BIP,
        "departure": DEP
    }
}
model = teqp.make_model(jconverted)

In [ ]:
%timeit teqp.make_model(jconverted)

From this example you can note that the first method is a lot slower because the FLD->JSON conversion needs to happen for each call, while in the second method it is much faster because only the JSON parsing needs to be done in ``teqp``.

In [ ]:
# It is also possible to prefix the path to indicate that the 
# indicated file (after the FLD::) should be converted from REFPROP format
jconverted = {
    "kind": "multifluid",
    "model": {
        "components": ["FLDPATH::R152A.FLD", 'FLDPATH::NEWR1234YF.FLD'],
        "BIP": BIP,
        "departure": DEP
    }
}
model = teqp.make_model(jconverted)

In [ ]:
%timeit teqp.make_model(jconverted)